영상 ./video/KETI_SL_0000006020.avi를 mediapipe로 처리하여 출력값 저장하기  
출력값은 Tensor나 list 저장

In [6]:
import mediapipe as mp
import numpy as np
import time
import cv2

In [2]:
# Prepare DrawingSpec
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Config holistic
mp_holistic = mp.solutions.holistic
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 영상 가져오기
cap = cv2.VideoCapture("./video/KETI_SL_0000006020.avi")

# 영상...
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# cap.set(cv2.CAP_PROP_FPS, 1)

total_frames = cap.get(7)
print(total_frames)

print(cap.get(cv2.CAP_PROP_FPS))

frame_rate = 10
prev = 0
i = 0
left_hand_lists = []
right_hand_lists = []
face_lists = []
pose_lists = []


while cap.isOpened():
#     time_elapsed = time.time() - prev
    success, image = cap.read()

#     if time_elapsed > 1./frame_rate:
#         prev = time.time()
        
    if not success:
        print("Ignoring empty camera frame.")
        # If loading a video, use 'break' instead of 'continue'.
        break

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    
    if results.left_hand_landmarks:
        left_hand_list = []
        for lm in results.left_hand_landmarks.landmark:
            left_hand_list.append([lm.x, lm.y, lm.z])
        left_hand_lists.append(left_hand_list)
        
    if results.right_hand_landmarks:
        right_hand_list = []
        for lm in results.right_hand_landmarks.landmark:
            right_hand_list.append([lm.x, lm.y, lm.z])
        right_hand_lists.append(right_hand_list)
        
    if results.face_landmarks:
        face_list = []
        for lm in results.face_landmarks.landmark:
            face_list.append([lm.x, lm.y, lm.z])
        face_lists.append(face_list)
        
    if results.pose_landmarks:
        pose_list = []
        for lm in results.pose_landmarks.landmark:
            pose_list.append([lm.x, lm.y, lm.z])
        pose_lists.append(pose_list)
        
        
#     cv2.imshow('MediaPipe Holistic', image)
#     if cv2.waitKey(5) & 0xFF == ord('q'):
#         break
holistic.close()
cap.release()
cv2.destroyAllWindows()

156.0
29.970029
Ignoring empty camera frame.


In [3]:
# 

import torch
lt = torch.FloatTensor(left_hand_lists)
rt = torch.FloatTensor(right_hand_lists)
ft = torch.FloatTensor(face_lists)
pt = torch.FloatTensor(pose_lists)

print(torch.FloatTensor(left_hand_lists).shape,
      torch.FloatTensor(right_hand_lists).shape,
      torch.FloatTensor(face_lists).shape,
      torch.FloatTensor(pose_lists).shape)

torch.Size([54, 21, 3]) torch.Size([0]) torch.Size([156, 468, 3]) torch.Size([156, 33, 3])


In [4]:
# 

torch.save(lt, 'tensor_lt.pt')
torch.save(rt, 'tensor_rt.pt')
torch.save(ft, 'tensor_ft.pt')
torch.save(pt, 'tensor_pt.pt')

In [5]:
print(torch.load('tensor_lt.pt'))

tensor([[[ 6.7036e-01,  8.9328e-01, -1.7920e-05],
         [ 6.5279e-01,  9.1216e-01, -1.2048e-02],
         [ 6.4243e-01,  9.4444e-01, -1.3254e-02],
         ...,
         [ 6.3723e-01,  9.9854e-01,  7.0965e-03],
         [ 6.2669e-01,  9.9468e-01,  5.2927e-03],
         [ 6.2288e-01,  9.8599e-01,  5.0059e-03]],

        [[ 6.4070e-01,  9.6747e-01,  5.9863e-05],
         [ 6.6071e-01,  9.4689e-01, -1.3350e-02],
         [ 6.7066e-01,  9.2707e-01, -3.7440e-02],
         ...,
         [ 6.1179e-01,  9.5497e-01, -7.8000e-02],
         [ 6.0643e-01,  9.4377e-01, -8.1491e-02],
         [ 6.0290e-01,  9.3101e-01, -8.5010e-02]],

        [[ 6.4161e-01,  8.1774e-01, -8.2350e-06],
         [ 6.1588e-01,  8.1947e-01, -4.9816e-03],
         [ 6.0007e-01,  8.1656e-01, -1.2252e-02],
         ...,
         [ 6.3402e-01,  8.0238e-01, -4.0941e-02],
         [ 6.3092e-01,  7.8710e-01, -4.4752e-02],
         [ 6.3087e-01,  7.7933e-01, -4.7794e-02]],

        ...,

        [[ 6.4063e-01,  8.0086e-01, -6